<a href="https://colab.research.google.com/github/revanth2802/text2mcqs/blob/main/text2mcqs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
# prject without transformers, no models used type-1
import re
import random

def generate_questions(text, num_questions=10):
    # Split text into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text.strip())
    sentences = [sentence for sentence in sentences if len(sentence.split()) > 5]  # Filter out short sentences

    # Generate questions
    questions = random.sample(sentences, min(num_questions, len(sentences)))
    return questions

def generate_mcqs(text):
    questions = generate_questions(text)

    mcqs = []
    for question in questions:
        # Extract keywords for options
        words = re.findall(r'\b\w+\b', question)
        random.shuffle(words)
        correct_answer = words[0]
        question_with_blank = re.sub(r'\b' + re.escape(correct_answer) + r'\b', '______', question, count=1)

        # Generate options
        options = [word for word in words if word != correct_answer]
        options = random.sample(options, min(3, len(options)))
        options.append(correct_answer)
        random.shuffle(options)

        # Format MCQ
        mcq = {
            "question": f"{question_with_blank.capitalize()}?",
            "options": options,
            "answer": correct_answer
        }
        mcqs.append(mcq)

    return mcqs

# Example usage
text = """
In the heart of a bustling city, amid the constant hum of traffic and the rhythmic pulse of daily life, lived a young artist named Mia. With her vibrant imagination and a heart full of dreams, she transformed her small apartment into a sanctuary of creativity. Her days were spent painting vivid murals on canvas, each stroke a reflection of her innermost thoughts and desires. One rainy afternoon, as the world outside was drenched in a melancholic downpour, Mia discovered an old, dusty journal hidden in a forgotten corner of her attic. Inside, she found a collection of letters and sketches from her grandmother, a renowned artist who had mysteriously vanished years ago. Intrigued and inspired, Mia embarked on a journey through the pages of the journal, uncovering a legacy of passion, mystery, and a hidden family secret that would change her life forever.
"""

mcqs = generate_mcqs(text)
for idx, mcq in enumerate(mcqs):
    print(f"Question {idx + 1}: {mcq['question']}")
    print("Options:")
    for option in mcq['options']:
        print(f"- {option}")
    print("Correct Answer:", mcq['answer'])
    print()


Question 1: One ______ afternoon, as the world outside was drenched in a melancholic downpour, mia discovered an old, dusty journal hidden in a forgotten corner of her attic.?
Options:
- rainy
- in
- dusty
- was
Correct Answer: rainy

Question 2: In the heart of a bustling city, amid the constant hum of traffic ______ the rhythmic pulse of daily life, lived a young artist named mia.?
Options:
- the
- and
- Mia
- the
Correct Answer: and

Question 3: With her vibrant imagination and a heart full of dreams, she transformed her small ______ into a sanctuary of creativity.?
Options:
- her
- she
- sanctuary
- apartment
Correct Answer: apartment

Question 4: Her days were spent painting vivid murals on ______, each stroke a reflection of her innermost thoughts and desires.?
Options:
- stroke
- canvas
- her
- spent
Correct Answer: canvas

Question 5: Intrigued and inspired, mia embarked on a journey through ______ pages of the journal, uncovering a legacy of passion, mystery, and a hidden fami

In [ ]:
# prject with transformers, no models used type-2
import random
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Initialize the model and tokenizer
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

text = """
In the quiet village of Eldenwood, nestled between ancient forests and rolling hills, lived a young blacksmith named Aiden. Known for his unparalleled skill and kind heart, Aiden crafted exquisite weapons and tools for his fellow villagers. One evening, as the sun dipped below the horizon, casting a golden glow over the village, a mysterious traveler arrived at Aiden's forge. The stranger, cloaked in shadows, presented Aiden with a peculiar, ancient sword that hummed with a strange energy. He spoke of an impending darkness that threatened to engulf Eldenwood and entrusted Aiden with a quest to reforge the sword with a rare, enchanted metal found only in the heart of the Forbidden Forest. With determination and a sense of destiny, Aiden set out on an adventure, facing mythical creatures, forging unlikely alliances, and uncovering hidden truths about his own heritage. As he journeyed deeper into the unknown, Aiden realized that the fate of Eldenwood rested not only on the strength of his arm but the courage of his spirit.
"""

def generate_mcqs(text, num_questions=10):
    questions = []

    # Split the text into sentences for better question extraction
    sentences = text.split('. ')

    # Generate questions using the T5 model
    for sentence in sentences:
        input_text = "generate question: " + sentence
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(input_ids, max_length=512, num_return_sequences=1, num_beams=5, no_repeat_ngram_size=2)

        question = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract a correct answer
        correct_answer = ""
        words = sentence.split()
        for word in words:
            if word.lower() in question.lower() and len(word) > 3:  # Avoid common short words
                correct_answer = word
                break

        if correct_answer:
            # Generate options
            options = random.sample([w for w in text.split() if w.lower() != correct_answer.lower() and len(w) > 3], 3)
            options.append(correct_answer)
            random.shuffle(options)  # Shuffle to randomize option order

            questions.append({
                "question": question.replace(correct_answer, "______") + "?",
                "options": options,
                "correct_answer": correct_answer
            })

        if len(questions) >= num_questions:
            break

    return questions

# Generate MCQs
mcqs = generate_mcqs(text)
for idx, mcq in enumerate(mcqs):
    print(f"Question {idx + 1}: {mcq['question']}")
    for opt_idx, option in enumerate(mcq['options']):
        print(f"Option {opt_idx + 1}: {option}")
    print(f"Correct Answer: {mcq['correct_answer']}")
    print("\n")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Question 1: a ______ blacksmith named Aiden?
Option 1: with
Option 2: skill
Option 3: horizon,
Option 4: young
Correct Answer: young


Question 2: ______?
Option 1: Aiden
Option 2: village,
Option 3: enchanted
Option 4: unlikely
Correct Answer: Aiden


Question 3: As the sun ______ below the horizon, casting a golden glow over the village,?
Option 1: below
Option 2: with
Option 3: dipped
Option 4: that
Correct Answer: dipped


Question 4: ______?
Option 1: quiet
Option 2: spirit.
Option 3: heart
Option 4: Aiden
Correct Answer: Aiden


Question 5: ______?
Option 1: peculiar,
Option 2: With
Option 3: Eldenwood
Option 4: forging
Correct Answer: Eldenwood


Question 6: ______ embarked on an adventure, facing mythical creatures, forging unlikely alliances, and uncovering hidden truths about his own heritage?
Option 1: creatures,
Option 2: Eldenwood
Option 3: Aiden
Option 4: with
Correct Answer: Aiden


Question 7: ______ realized that the fate of Eldenwood rested on the strength of his arm?